In [33]:
import tensorflow as tf
from tensorflow.keras import layers
import json
import numpy as np

num_classes = 6
with open('training.json', 'r') as f:
    data_dict = json.load(f)

obj = {
    "Hand Close": 0,
    "Thumb": 1,
    "Index": 2,
    "Middle": 3,
    "Ring": 4,
    "Pinky": 5
}
    
# extract the x and y data from the JSON dictionary
x_data = [[int(x) for x in data['xs'].values()] for data in data_dict['data']]
y_data = [obj[data['ys']['label']] for data in data_dict['data']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_data_norm = scaler.fit_transform(x_data)
#print(x_data_norm)

x_data = np.array(x_data_norm)
y_data = np.array(y_data)

y_data_c = tf.keras.utils.to_categorical(y_data, num_classes)

print(x_data, y_data_c)

#from sklearn.preprocessing import LabelEncoder

#labels = ['Hand Close', 'Thumb', 'Index', 'Middle', 'Ring', 'Pinky']

#encoder = LabelEncoder()
#int_labels = encoder.fit_transform(labels)

#print(encoder, int_labels)


model = tf.keras.Sequential([
    tf.keras.layers.Dense(58, activation='relu', input_shape=(999,)),
    tf.keras.layers.Dense(58, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

# compile the model with a categorical cross-entropy loss function
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train the model
model.fit(x_data, y_data_c, epochs=200, batch_size=32)

with open('testing.json', 'r') as f:
    data_dict = json.load(f)
    
x_test = [[int(x) for x in data['xs'].values()] for data in data_dict['data']]
y_test = [obj[data['ys']['label']] for data in data_dict['data']]
y_test_c = tf.keras.utils.to_categorical(y_test, num_classes)
x_test_norm = scaler.fit_transform(x_test)
print(x_test_norm, y_test_c)
predictions = model.predict(x_test_norm)
for i in range(len(predictions)):
    pred_label = np.argmax(predictions[i])
    true_label = y_test[i]
    print(f"Prediction: {pred_label}, True Label: {true_label}")
    
accuracy = model.evaluate(x_test_norm, y_test_c)[1]
print(f"Accuracy: {accuracy}")

[[0.74025974 0.89095745 0.86197917 ... 0.27272727 0.42253521 0.50632911]
 [0.72294372 0.91223404 0.8828125  ... 0.25974026 0.42253521 0.51898734]
 [0.73160173 0.90691489 0.89322917 ... 0.33766234 0.46478873 0.50632911]
 ...
 [0.73376623 0.8962766  0.875      ... 0.37662338 0.42253521 0.44303797]
 [0.72727273 0.89361702 0.8671875  ... 0.2987013  0.50704225 0.44303797]
 [0.73809524 0.89095745 0.88020833 ... 0.31168831 0.43661972 0.40506329]] [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1.

4/4 [==============================] - 0s 5ms/step - loss: 0.6166 - accuracy: 0.8824
Epoch 142/200
4/4 [==============================] - 0s 5ms/step - loss: 0.6014 - accuracy: 0.8922
Epoch 143/200
4/4 [==============================] - 0s 4ms/step - loss: 0.6071 - accuracy: 0.8627
Epoch 144/200
4/4 [==============================] - 0s 5ms/step - loss: 0.6392 - accuracy: 0.8235
Epoch 145/200
4/4 [==============================] - 0s 5ms/step - loss: 0.7242 - accuracy: 0.7843
Epoch 146/200
4/4 [==============================] - 0s 4ms/step - loss: 0.6042 - accuracy: 0.8627
Epoch 147/200
4/4 [==============================] - 0s 3ms/step - loss: 0.6016 - accuracy: 0.8431
Epoch 148/200
4/4 [==============================] - 0s 4ms/step - loss: 0.6174 - accuracy: 0.8529
Epoch 149/200
4/4 [==============================] - 0s 5ms/step - loss: 0.5909 - accuracy: 0.8333
Epoch 150/200
4/4 [==============================] - 0s 5ms/step - loss: 0.5883 - accuracy: 0.8431
Epoch 151/200
4/4 [=====

In [39]:
import json
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# load the JSON data
num_classes = 6
with open('training.json', 'r') as f:
    data_dict = json.load(f)

obj = {
    "Hand Close": 0,
    "Thumb": 1,
    "Index": 2,
    "Middle": 3,
    "Ring": 4,
    "Pinky": 5
}
    
# extract the x and y data from the JSON dictionary
xs_train = [[int(x) for x in data['xs'].values()] for data in data_dict['data']]
ys_train = [obj[data['ys']['label']] for data in data_dict['data']]
ys_train = tf.keras.utils.to_categorical(ys_train, num_classes)

xs_train = np.array(xs_train)
ys_train = np.array(ys_train)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xs_train, ys_train, test_size = 0.25, random_state = 0) 

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X = scaler.transform(xs_train)

# define the number of folds
k = 5

# create a KFold object
kf = KFold(n_splits=k, shuffle=True)

# iterate over the folds
for fold, (train_index, test_index) in enumerate(kf.split(x_data_norm)):
    # get the training and test data for this fold
    x_train, x_test = x_data_norm[train_index], x_data_norm[test_index]
    y_train, y_test = y_data_c[train_index], y_data_c[test_index]

    # normalize the data
    x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
    x_test = (x_test - x_train.min()) / (x_train.max() - x_train.min())

    # create the model
    model = Sequential([
        tf.keras.layers.Dense(58, activation='relu', input_shape=(999,)),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        #tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        #tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(num_classes, activation='softmax'),
    ])

    # compile the model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    # train the model on the training data
    model.fit(x_train, y_train, epochs=1000, batch_size=1000, verbose=0)

    # evaluate the model on the test data
    loss, accuracy = model.evaluate(x_test, y_test, batch_size=32, verbose=0)

    # print the test accuracy for this fold
    print(f'Fold {fold+1} Test accuracy: {accuracy:.4f}')


Fold 1 Test accuracy: 0.1429
Fold 2 Test accuracy: 0.2381
Fold 3 Test accuracy: 0.0500
Fold 4 Test accuracy: 0.2000
Fold 5 Test accuracy: 0.3500


In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import  SGD
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scikeras.wrappers import KerasClassifier

# define the input shape
input_shape = (999,)
num_classes = 6

with open('training.json', 'r') as f:
    data_dict = json.load(f)

obj = {
    "Hand Close": 0,
    "Thumb": 1,
    "Index": 2,
    "Middle": 3,
    "Ring": 4,
    "Pinky": 5
}
    
# extract the x and y data from the JSON dictionary
xs_train = [[int(x) for x in data['xs'].values()] for data in data_dict['data']]
ys_train = [obj[data['ys']['label']] for data in data_dict['data']]
ys_train = tf.keras.utils.to_categorical(ys_train, num_classes)

xs_train = np.array(xs_train)
ys_train = np.array(ys_train)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xs_train, ys_train, test_size = 0.25, random_state = 0) 

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X = scaler.transform(xs_train)


model = Sequential()
model.add(Dense(58, input_shape=input_shape, activation='relu'))
model.add(Dropout(0.1)),
model.add(Dense(58, activation='relu')),
model.add(Dropout(0.1)),
model.add(Dense(58, activation='relu')),
model.add(Dropout(0.1)),
model.add(Dense(58, activation='relu')),
model.add(Dropout(0.1)),
model.add(Dense(58, activation='relu')),
#model.add(Dropout(0.1)),
model.add(Dense(58, activation='relu')),
#model.add(Dropout(0.1)),
model.add(Dense(num_classes, activation='softmax')),
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1000, batch_size=1000)

i = 0
while i < 0:
    y_pred = model.predict(X_test)
    y_pred_thres = (y_pred > 1)
    y_pred_thres = y_pred_thres * 1
    y_pred2 = y_pred_thres.argmax(axis=1)
    y_test2 = y_test.argmax(axis=1)
    from sllearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred2)
    
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test2, y_pred2)
    print(f'accuracyis: {acuuracy} when threshold is: {i}')
    i = i + 0.05

y_pred = model.predict(X_test)
y_pred_thres = (y_pred > 0.15)
y_pred_thres = y_pred_thres * 1
y_pred2 = y_pred_thres.argmax(axis=1)
y_test2 = y_test.argmax(axis=1)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test2, y_pred2) 

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test2, y_pred2)

print(f'accuracy is: {accuracy}')

def baseline_model():
    model = Sequential()
    model.add(Dense(58, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.1)),
    model.add(Dense(58, activation='relu')),
    model.add(Dropout(0.1)),
    model.add(Dense(58, activation='relu')),
    model.add(Dropout(0.1)),
    model.add(Dense(58, activation='relu')),
    model.add(Dropout(0.1)),
    model.add(Dense(58, activation='relu')),
    #model.add(Dropout(0.1)),
    model.add(Dense(58, activation='relu')),
    #model.add(Dropout(0.1)),
    model.add(Dense(num_classes, activation='softmax')),
    sgd = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

estimator = KerasClassifier(model=baseline_model, epochs=1000, batch_size=100)

kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, xs_train, ys_train, cv=kfold)
print("Baseline: %.2f (5.2f%%)" % (results.mean() * 100, results.std() * 100))

In [52]:
from scikeras.wrappers import KerasClassifier

In [49]:
pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [56]:
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import  SGD
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scikeras.wrappers import KerasClassifier

# define the input shape
input_shape = (999,)
num_classes = 6

with open('training.json', 'r') as f:
    data_dict = json.load(f)

obj = {
    "Hand Close": 0,
    "Thumb": 1,
    "Index": 2,
    "Middle": 3,
    "Ring": 4,
    "Pinky": 5
}
    
# extract the x and y data from the JSON dictionary
xs_train = [[int(x) for x in data['xs'].values()] for data in data_dict['data']]
ys_train = [obj[data['ys']['label']] for data in data_dict['data']]
ys_train = tf.keras.utils.to_categorical(ys_train, num_classes)

xs_train = np.array(xs_train)
ys_train = np.array(ys_train)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xs_train, ys_train, test_size = 0.25, random_state = 0) 

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X = scaler.transform(xs_train)

dim = 999
dropout = 0.1

model = Sequential()
model.add(Dense(dim, input_shape=input_shape, activation='relu'))
model.add(Dropout(dropout)),
model.add(Dense(dim, activation='relu')),
model.add(Dropout(dropout)),
model.add(Dense(dim, activation='relu')),
model.add(Dropout(dropout)),
model.add(Dense(dim, activation='relu')),
model.add(Dropout(dropout)),
model.add(Dense(dim, activation='relu')),
#model.add(Dropout(dropout)),
model.add(Dense(dim, activation='relu')),
#model.add(Dropout(dropout)),
model.add(Dense(num_classes, activation='softmax')),
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1000, batch_size=1000)

i = 0
while i < 0:
    y_pred = model.predict(X_test)
    y_pred_thres = (y_pred > 1)
    y_pred_thres = y_pred_thres * 1
    y_pred2 = y_pred_thres.argmax(axis=1)
    y_test2 = y_test.argmax(axis=1)
    from sllearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred2)
    
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test2, y_pred2)
    print(f'accuracyis: {acuuracy} when threshold is: {i}')
    i = i + 0.05

y_pred = model.predict(X_test)
y_pred_thres = (y_pred > 0.15)
y_pred_thres = y_pred_thres * 1
y_pred2 = y_pred_thres.argmax(axis=1)
y_test2 = y_test.argmax(axis=1)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test2, y_pred2) 

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test2, y_pred2)

print(f'accuracy is: {accuracy}')

def baseline_model():
    model = Sequential()
    model.add(Dense(dim, input_shape=input_shape, activation='relu'))
    model.add(Dropout(dropout)),
    model.add(Dense(dim, activation='relu')),
    model.add(Dropout(dropout)),
    model.add(Dense(dim, activation='relu')),
    model.add(Dropout(dropout)),
    model.add(Dense(dim, activation='relu')),
    model.add(Dropout(dropout)),
    model.add(Dense(dim, activation='relu')),
    #model.add(Dropout(dropout)),
    model.add(Dense(dim, activation='relu')),
    #model.add(Dropout(dropout)),
    model.add(Dense(num_classes, activation='softmax')),
    sgd = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

estimator = KerasClassifier(model=baseline_model, epochs=1000, batch_size=100)

kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, xs_train, ys_train, cv=kfold)
print("Baseline: %.2f (5.2f%%)" % (results.mean() * 100, results.std() * 100))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 1.8183 - accuracy: 0.1447
Epoch 2/1000
1/1 [==============================] - 0s 107ms/step - loss: 2.1377 - accuracy: 0.2105
Epoch 3/1000
1/1 [==============================] - 0s 126ms/step - loss: 1.9003 - accuracy: 0.1842
Epoch 4/1000
1/1 [==============================] - 0s 115ms/step - loss: 1.8135 - accuracy: 0.1974
Epoch 5/1000
1/1 [==============================] - 0s 111ms/step - loss: 1.7926 - accuracy: 0.1974
Epoch 6/1000
1/1 [==============================] - 0s 106ms/step - loss: 1.7712 - accuracy: 0.3026
Epoch 7/1000
1/1 [==============================] - 0s 122ms/step - loss: 1.7751 - accuracy: 0.2237
Epoch 8/1000
1/1 [==============================] - 0s 109ms/step - loss: 1.7780 - accuracy: 0.2105
Epoch 9/1000
1/1 [==============================] - 0s 105ms/step - loss: 1.7851 - accuracy: 0.1711
Epoch 10/1000
1/1 [==============================] - 0s 122ms/step - loss: 1.7752 - accuracy: 0.2763
Ep

1/1 [==============================] - 0s 110ms/step - loss: 0.7469 - accuracy: 0.6842
Epoch 83/1000
1/1 [==============================] - 0s 128ms/step - loss: 0.7732 - accuracy: 0.6842
Epoch 84/1000
1/1 [==============================] - 0s 113ms/step - loss: 0.8091 - accuracy: 0.6711
Epoch 85/1000
1/1 [==============================] - 0s 109ms/step - loss: 0.9054 - accuracy: 0.6447
Epoch 86/1000
1/1 [==============================] - 0s 111ms/step - loss: 1.2468 - accuracy: 0.4868
Epoch 87/1000
1/1 [==============================] - 0s 108ms/step - loss: 1.0088 - accuracy: 0.6447
Epoch 88/1000
1/1 [==============================] - 0s 107ms/step - loss: 2.0002 - accuracy: 0.4342
Epoch 89/1000
1/1 [==============================] - 0s 119ms/step - loss: 0.9618 - accuracy: 0.6316
Epoch 90/1000
1/1 [==============================] - 0s 115ms/step - loss: 1.4236 - accuracy: 0.4605
Epoch 91/1000
1/1 [==============================] - 0s 106ms/step - loss: 1.2506 - accuracy: 0.5000
Epoc

1/1 [==============================] - 0s 118ms/step - loss: 1.0665 - accuracy: 0.6053
Epoch 163/1000
1/1 [==============================] - 0s 123ms/step - loss: 0.8480 - accuracy: 0.6447
Epoch 164/1000
1/1 [==============================] - 0s 120ms/step - loss: 0.9008 - accuracy: 0.6842
Epoch 165/1000
1/1 [==============================] - 0s 126ms/step - loss: 0.8089 - accuracy: 0.6842
Epoch 166/1000
1/1 [==============================] - 0s 112ms/step - loss: 0.8878 - accuracy: 0.6447
Epoch 167/1000
1/1 [==============================] - 0s 113ms/step - loss: 0.8693 - accuracy: 0.6447
Epoch 168/1000
1/1 [==============================] - 0s 118ms/step - loss: 0.6908 - accuracy: 0.7500
Epoch 169/1000
1/1 [==============================] - 0s 115ms/step - loss: 0.9344 - accuracy: 0.6184
Epoch 170/1000
1/1 [==============================] - 0s 112ms/step - loss: 0.8935 - accuracy: 0.6447
Epoch 171/1000
1/1 [==============================] - 0s 112ms/step - loss: 0.7282 - accuracy: 0.

1/1 [==============================] - 0s 117ms/step - loss: 0.4722 - accuracy: 0.8158
Epoch 243/1000
1/1 [==============================] - 0s 126ms/step - loss: 0.1465 - accuracy: 0.9474
Epoch 244/1000
1/1 [==============================] - 0s 124ms/step - loss: 0.2264 - accuracy: 0.8947
Epoch 245/1000
1/1 [==============================] - 0s 119ms/step - loss: 0.3310 - accuracy: 0.7895
Epoch 246/1000
1/1 [==============================] - 0s 123ms/step - loss: 0.3324 - accuracy: 0.8816
Epoch 247/1000
1/1 [==============================] - 0s 139ms/step - loss: 0.3084 - accuracy: 0.8684
Epoch 248/1000
1/1 [==============================] - 0s 118ms/step - loss: 0.2637 - accuracy: 0.8947
Epoch 249/1000
1/1 [==============================] - 0s 125ms/step - loss: 0.2115 - accuracy: 0.9342
Epoch 250/1000
1/1 [==============================] - 0s 127ms/step - loss: 0.2962 - accuracy: 0.8684
Epoch 251/1000
1/1 [==============================] - 0s 119ms/step - loss: 0.3803 - accuracy: 0.

1/1 [==============================] - 0s 114ms/step - loss: 0.7113 - accuracy: 0.7500
Epoch 323/1000
1/1 [==============================] - 0s 118ms/step - loss: 0.6456 - accuracy: 0.7632
Epoch 324/1000
1/1 [==============================] - 0s 115ms/step - loss: 0.6913 - accuracy: 0.7368
Epoch 325/1000
1/1 [==============================] - 0s 121ms/step - loss: 0.8041 - accuracy: 0.7632
Epoch 326/1000
1/1 [==============================] - 0s 113ms/step - loss: 0.7630 - accuracy: 0.7105
Epoch 327/1000
1/1 [==============================] - 0s 121ms/step - loss: 0.8708 - accuracy: 0.7237
Epoch 328/1000
1/1 [==============================] - 0s 121ms/step - loss: 0.4996 - accuracy: 0.8421
Epoch 329/1000
1/1 [==============================] - 0s 117ms/step - loss: 0.7154 - accuracy: 0.6579
Epoch 330/1000
1/1 [==============================] - 0s 115ms/step - loss: 0.6530 - accuracy: 0.7763
Epoch 331/1000
1/1 [==============================] - 0s 114ms/step - loss: 0.4808 - accuracy: 0.

...

1/1 [==============================] - 0s 63ms/step - loss: nan - accuracy: 0.1585
Epoch 697/1000
1/1 [==============================] - 0s 71ms/step - loss: nan - accuracy: 0.1585
Epoch 698/1000
1/1 [==============================] - 0s 71ms/step - loss: nan - accuracy: 0.1585
Epoch 699/1000
1/1 [==============================] - 0s 69ms/step - loss: nan - accuracy: 0.1585
Epoch 700/1000
1/1 [==============================] - 0s 73ms/step - loss: nan - accuracy: 0.1585
Epoch 701/1000
1/1 [==============================] - 0s 67ms/step - loss: nan - accuracy: 0.1585
Epoch 702/1000
1/1 [==============================] - 0s 71ms/step - loss: nan - accuracy: 0.1585
Epoch 703/1000
1/1 [==============================] - 0s 80ms/step - loss: nan - accuracy: 0.1585
Epoch 704/1000
1/1 [==============================] - 0s 72ms/step - loss: nan - accuracy: 0.1585
Epoch 705/1000
1/1 [==============================] - 0s 63ms/step - loss: nan - accuracy: 0.1585
Epoch 706/1000
1/1 [===============

1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 780/1000
1/1 [==============================] - 0s 75ms/step - loss: nan - accuracy: 0.1585
Epoch 781/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 782/1000
1/1 [==============================] - 0s 69ms/step - loss: nan - accuracy: 0.1585
Epoch 783/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 784/1000
1/1 [==============================] - 0s 67ms/step - loss: nan - accuracy: 0.1585
Epoch 785/1000
1/1 [==============================] - 0s 69ms/step - loss: nan - accuracy: 0.1585
Epoch 786/1000
1/1 [==============================] - 0s 67ms/step - loss: nan - accuracy: 0.1585
Epoch 787/1000
1/1 [==============================] - 0s 76ms/step - loss: nan - accuracy: 0.1585
Epoch 788/1000
1/1 [==============================] - 0s 78ms/step - loss: nan - accuracy: 0.1585
Epoch 789/1000
1/1 [===============

1/1 [==============================] - 0s 63ms/step - loss: nan - accuracy: 0.1585
Epoch 863/1000
1/1 [==============================] - 0s 68ms/step - loss: nan - accuracy: 0.1585
Epoch 864/1000
1/1 [==============================] - 0s 75ms/step - loss: nan - accuracy: 0.1585
Epoch 865/1000
1/1 [==============================] - 0s 72ms/step - loss: nan - accuracy: 0.1585
Epoch 866/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 867/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 868/1000
1/1 [==============================] - 0s 73ms/step - loss: nan - accuracy: 0.1585
Epoch 869/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 870/1000
1/1 [==============================] - 0s 68ms/step - loss: nan - accuracy: 0.1585
Epoch 871/1000
1/1 [==============================] - 0s 73ms/step - loss: nan - accuracy: 0.1585
Epoch 872/1000
1/1 [===============

1/1 [==============================] - 0s 63ms/step - loss: nan - accuracy: 0.1585
Epoch 946/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 947/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 948/1000
1/1 [==============================] - 0s 68ms/step - loss: nan - accuracy: 0.1585
Epoch 949/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 950/1000
1/1 [==============================] - 0s 71ms/step - loss: nan - accuracy: 0.1585
Epoch 951/1000
1/1 [==============================] - 0s 68ms/step - loss: nan - accuracy: 0.1585
Epoch 952/1000
1/1 [==============================] - 0s 70ms/step - loss: nan - accuracy: 0.1585
Epoch 953/1000
1/1 [==============================] - 0s 71ms/step - loss: nan - accuracy: 0.1585
Epoch 954/1000
1/1 [==============================] - 0s 69ms/step - loss: nan - accuracy: 0.1585
Epoch 955/1000
1/1 [===============

TypeError: not all arguments converted during string formatting